<a href="https://colab.research.google.com/github/dvaughanCogniCom/23XI/blob/main/23XI_dvaughan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

uploaded = files.upload()


KeyboardInterrupt: ignored

In [4]:
import pandas as pd
import numpy as np
import io

df_location = pd.read_csv(io.BytesIO(uploaded['final_location.csv']))
df_pitstops = pd.read_csv(io.BytesIO(uploaded['final_pit.csv']))


Change record_timestamp column to datetime milliseconds, and cast all integer metrics to floats (for interpolation)

In [5]:
df_location['record_timestamp']=pd.to_datetime(df_location['record_timestamp'],format='%Y-%m-%d %H:%M:%S.%f').astype(int)/10**6
df_location['record_timestamp']=df_location['record_timestamp'].astype(int)
df_location['brake']=df_location['brake'].astype(float)
df_location['throttle']=df_location['throttle'].astype(float)
df_location['rpm']=df_location['rpm'].astype(float)
df_location['lap']=df_location['lap_frac'].astype(int)
df_location['lap']=df_location['lap'].astype(float)
df_location.describe()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  """Entry point for launching an IPython kernel.


,record_timestamp,car,throttle,speed,steering,longitude,latitude,rpm,brake,lap_frac,lap
count,2.578760e+05,257876.000000,257876.000000,257876.000000,257876.000000,257876.000000,257876.000000,257876.000000,257876.000000,257876.000000,257876.000000
mean,1.572204e+12,18.464173,39.358893,73.673254,1.442345,-79.851653,36.634111,5805.064438,108.627267,30.525161,30.012770
std,4.086490e+05,10.965721,37.929253,25.692809,3.034708,0.000703,0.000972,1734.143023,173.305087,16.318492,16.321293
min,1.572204e+12,0.000000,0.000000,0.000000,-87.300000,-79.852790,36.632618,0.000000,0.000000,0.000112,0.000000
25%,1.572204e+12,9.000000,6.000000,60.224000,-0.200000,-79.852290,36.633164,4684.000000,3.000000,17.684662,17.000000
50%,1.572205e+12,18.000000,18.000000,72.529000,0.200000,-79.851654,36.634150,5617.000000,10.000000,30.690175,30.000000
75%,1.572205e+12,28.000000,90.000000,93.925000,3.000000,-79.851010,36.635060,7175.000000,145.000000,43.569147,43.000000
max,1.572205e+12,37.000000,100.000000,121.474000,22.400000,-79.850540,36.635540,9652.000000,1463.000000,59.999966,59.000000


In [6]:
df_location.head()

,record_timestamp,car,throttle,speed,steering,longitude,latitude,rpm,brake,lap_frac,lap
0,1572203804631,29,9.0,64.326,0.0,-79.85263,36.632855,4818.0,24.0,1.203817,1.0
1,1572203841545,29,100.0,88.115,0.0,-79.85195,36.634920,6918.0,14.0,2.905288,2.0
2,1572204145686,29,9.0,101.650,0.0,-79.85263,36.633650,7281.0,493.0,17.085224,17.0
3,1572204339609,29,9.0,107.666,0.0,-79.85257,36.633774,7650.0,527.0,26.068089,26.0
4,1572204462055,29,25.0,46.416,0.0,-79.85143,36.633217,3685.0,11.0,29.400984,29.0


Find caution laps in data and remove them

In [7]:
df_laps = df_location.copy()

def check_for_caution(group):    
  if ((group.speed.max() < 80) or (group.speed.min()<50)):
    group=group.assign(lap_type='pit_stop')
  elif group.lap.max()<31:
    group=group.assign(lap_type='stint_1')
  elif group.lap.max()>31:
    group=group.assign(lap_type='stint_2')
  return group  

df_laps['lap_type']=None
df_laps=df_laps.groupby(['car','lap']).apply(lambda x : check_for_caution(x))
df_laps.reset_index(drop=True,inplace=True)
df_laps.loc[(df_laps['car']==0) & (df_laps['lap']>=34) & (df_laps['lap']<35)]
                                   
                                   

,record_timestamp,car,throttle,speed,steering,longitude,latitude,rpm,brake,lap_frac,lap,lap_type
4000,1572204658012,0,8.0,59.404,7.9,-79.851030,36.635467,4355.0,8.0,34.757015,34.0,stint_2
4001,1572204656949,0,0.0,63.710,7.8,-79.850770,36.635353,4794.0,0.0,34.716400,34.0,stint_2
4002,1572204648967,0,64.0,65.966,4.0,-79.851930,36.632730,5304.0,5.0,34.308514,34.0,stint_2
4003,1572204661716,0,99.0,96.181,-1.4,-79.852030,36.634747,7616.0,5.0,34.928770,34.0,stint_2
4004,1572204644185,0,0.0,86.816,1.5,-79.852680,36.633434,6274.0,313.0,34.112850,34.0,stint_2
...,...,...,...,...,...,...,...,...,...,...,...,...
4101,1572204657465,0,0.0,62.138,8.1,-79.850860,36.635414,4663.0,3.0,34.732536,34.0,stint_2
4102,1572204656403,0,0.0,67.607,5.6,-79.850660,36.635223,5044.0,105.0,34.690080,34.0,stint_2
4103,1572204651622,0,99.0,99.804,-1.5,-79.851270,36.633500,7840.0,3.0,34.441650,34.0,stint_2
4104,1572204662527,0,99.0,103.769,-1.6,-79.852150,36.634520,8110.0,3.0,34.961067,34.0,stint_2


In [8]:
df_laps['lap_frac_only']=df_laps['lap_frac']-df_laps['lap']

min_time_stint_1 = df_laps.loc[df_laps['lap_type']=='stint_1', 'record_timestamp'].min()
min_time_stint_2 = df_laps.loc[df_laps['lap_type']=='stint_2', 'record_timestamp'].min()

min_lap_stint_1 = df_laps.loc[df_laps['lap_type']=='stint_1', 'lap'].min()
max_lap_stint_1 = df_laps.loc[df_laps['lap_type']=='stint_1', 'lap'].max()
min_lap_stint_2 = df_laps.loc[df_laps['lap_type']=='stint_2', 'lap'].min()
max_lap_stint_2 = df_laps.loc[df_laps['lap_type']=='stint_2', 'lap'].max()
print(f'stint 1 first lap: {min_lap_stint_1}')
print(f'stint 1 last lap: {max_lap_stint_1}')
print(f'stint 1 min time: {min_time_stint_1}')

print(f'stint 2 first lap: {min_lap_stint_2}')
print(f'stint 2 last lap: {max_lap_stint_2}')
print(f'stint 2 min time: {min_time_stint_2}')


stint 1 first lap: 0.0
stint 1 last lap: 27.0
stint 1 min time: 1572203772380
stint 2 first lap: 32.0
stint 2 last lap: 59.0
stint 2 min time: 1572204587969


In [9]:
df_laps.loc[df_laps['lap_type']=='stint_2', 'lap'] = df_laps.loc[df_laps['lap_type']=='stint_2', 'lap']-min_lap_stint_2

df_laps.loc[df_laps['lap_type']=='stint_2', 'stint_time'] = (df_laps.loc[df_laps['lap_type']=='stint_2', 'record_timestamp']-min_time_stint_2)/1000
df_laps.loc[df_laps['lap_type']=='stint_1', 'stint_time'] = (df_laps.loc[df_laps['lap_type']=='stint_1', 'record_timestamp']-min_time_stint_1)/1000

In [10]:
df_stints = df_laps.loc[df_laps['lap_type']!='pit_stop']
df_stints.describe()

,record_timestamp,car,throttle,speed,steering,longitude,latitude,rpm,brake,lap_frac,lap,lap_frac_only,stint_time
count,2.019810e+05,201981.000000,201981.000000,201981.000000,201981.000000,201981.000000,201981.000000,201981.000000,201981.000000,201981.000000,201981.000000,201981.000000,201981.000000
mean,1.572204e+12,18.430318,45.907511,83.306635,1.600399,-79.851653,36.634111,6355.708671,124.717305,30.513362,13.725444,0.512485,308.271573
std,4.594489e+05,10.953748,39.653181,18.290767,2.916708,0.000710,0.000986,1372.627072,185.785615,18.411998,7.818780,0.285081,164.699366
min,1.572204e+12,0.000000,0.000000,50.244000,-23.000000,-79.852790,36.632618,3636.000000,0.000000,0.000112,0.000000,0.000000,0.000000
25%,1.572204e+12,9.000000,7.000000,66.376000,0.000000,-79.852295,36.633137,5088.000000,3.000000,13.793384,7.000000,0.261665,166.591000
50%,1.572205e+12,18.000000,37.000000,80.595000,0.500000,-79.851650,36.634155,6153.000000,10.000000,34.275680,14.000000,0.522286,308.902000
75%,1.572205e+12,28.000000,91.000000,99.052000,3.300000,-79.851000,36.635082,7527.000000,212.000000,47.132670,21.000000,0.762821,450.346000
max,1.572205e+12,37.000000,100.000000,121.474000,13.000000,-79.850550,36.635540,9652.000000,918.000000,59.999966,27.000000,0.999983,638.851000


In [11]:
df_stints.loc[df_stints['lap_type']=='stint_1'].describe()

,record_timestamp,car,throttle,speed,steering,longitude,latitude,rpm,brake,lap_frac,lap,lap_frac_only,stint_time
count,9.925200e+04,99252.000000,99252.000000,99252.000000,99252.000000,99252.000000,99252.000000,99252.000000,99252.000000,99252.000000,99252.000000,99252.000000,99252.000000
mean,1.572204e+12,18.359821,46.016020,83.419201,1.576996,-79.851642,36.634136,6365.464827,124.629156,13.544031,13.024574,0.519457,286.165042
std,1.673431e+05,10.947120,39.633486,18.296534,2.937798,0.000708,0.000984,1372.862576,186.091589,8.018980,8.009203,0.285170,167.343100
min,1.572204e+12,0.000000,0.000000,50.244000,-23.000000,-79.852780,36.632626,3636.000000,0.000000,0.000112,0.000000,0.000000,0.000000
25%,1.572204e+12,9.000000,7.000000,66.513000,-0.100000,-79.852280,36.633164,5100.000000,3.000000,6.515723,6.000000,0.267788,139.695000
50%,1.572204e+12,18.000000,37.000000,80.732000,0.500000,-79.851640,36.634212,6162.000000,10.000000,13.581425,13.000000,0.538768,286.122500
75%,1.572204e+12,28.000000,90.000000,99.189000,3.300000,-79.850990,36.635100,7537.000000,212.000000,20.527281,20.000000,0.768099,431.161250
max,1.572204e+12,37.000000,100.000000,121.474000,13.000000,-79.850550,36.635540,9535.000000,902.000000,27.999796,27.000000,0.999983,600.573000


In [12]:
df_stints.loc[df_stints['lap_type']=='stint_2'].describe()

,record_timestamp,car,throttle,speed,steering,longitude,latitude,rpm,brake,lap_frac,lap,lap_frac_only,stint_time
count,1.027290e+05,102729.000000,102729.000000,102729.000000,102729.000000,102729.000000,102729.000000,102729.000000,102729.000000,102729.000000,102729.000000,102729.000000,102729.000000
mean,1.572205e+12,18.498428,45.802675,83.197878,1.623010,-79.851664,36.634085,6346.282724,124.802471,46.908343,14.402593,0.505750,329.629879
std,1.592158e+05,10.959770,39.672111,18.284624,2.896022,0.000712,0.000987,1372.340308,185.490383,7.575779,7.568935,0.284835,159.215831
min,1.572205e+12,0.000000,0.000000,52.363000,-10.900000,-79.852790,36.632618,3848.000000,0.000000,32.001590,0.000000,0.000015,0.000000
25%,1.572205e+12,9.000000,6.000000,66.240000,0.000000,-79.852310,36.633110,5077.000000,3.000000,40.338337,8.000000,0.256206,191.927000
50%,1.572205e+12,18.000000,38.000000,80.527000,0.500000,-79.851660,36.634098,6143.000000,10.000000,46.881954,14.000000,0.505997,329.136000
75%,1.572205e+12,28.000000,91.000000,98.984000,3.300000,-79.851010,36.635060,7515.000000,212.000000,53.495890,21.000000,0.757107,467.503000
max,1.572205e+12,37.000000,100.000000,121.269000,12.600000,-79.850550,36.635530,9652.000000,918.000000,59.999966,27.000000,0.999980,638.851000


Modify data for training

In [13]:
df_stints.tail()

,record_timestamp,car,throttle,speed,steering,longitude,latitude,rpm,brake,lap_frac,lap,lap_type,lap_frac_only,stint_time
257871,1572205169375,37,26.0,57.695,7.5,-79.85225,36.632667,4482.0,0.0,59.264560,27.0,stint_2,0.264560,581.406
257872,1572205182387,37,85.0,77.382,1.6,-79.85177,36.635166,6214.0,0.0,59.867798,27.0,stint_2,0.867798,594.418
257873,1572205177605,37,0.0,72.529,5.0,-79.85062,36.635150,5327.0,149.0,59.677086,27.0,stint_2,0.677086,589.636
257874,1572205179465,37,10.0,56.464,7.4,-79.85098,36.635464,4401.0,0.0,59.751050,27.0,stint_2,0.751050,591.496
257875,1572205171230,37,99.0,74.580,2.9,-79.85168,36.632893,5930.0,0.0,59.351600,27.0,stint_2,0.351600,583.261


In [14]:
from sklearn.utils import shuffle
max_stint_laps = 40
df_input = df_stints.copy()
df_input = df_input.dropna()
df_target = df_input['stint_time']

df_input.drop(columns=['record_timestamp','throttle','speed','steering','longitude','latitude','rpm','brake','lap_frac','stint_time'],inplace=True)
df_input.rename(columns={'car':'cat_car','lap_type':'cat_stint_number','lap_frac_only':'num_lap_frac_only','lap':'num_lap'},inplace=True)
#df_input['num_lap']=df_input['num_lap'].copy()/max_stint_laps

df_input,df_target = shuffle(df_input,df_target, random_state=0)
df_input.tail()

,cat_car,num_lap,cat_stint_number,num_lap_frac_only
225312,33,16.0,stint_1,0.931112
150028,22,0.0,stint_1,0.316725
222034,32,11.0,stint_2,0.103577
55327,8,11.0,stint_1,0.642366
255235,37,2.0,stint_2,0.946220


Make data selectors

In [15]:
from sklearn.compose import make_column_selector

cat_selector = make_column_selector(pattern='^cat_')
num_selector = make_column_selector(pattern='^num_')

print(f'categories {cat_selector(df_input)}')
print(f'numbers {num_selector(df_input)}')

categories ['cat_car', 'cat_stint_number']
numbers ['num_lap', 'num_lap_frac_only']


Create Linear Regressor Pipeline

In [16]:
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.svm import SVR

cat_linear_processor = OneHotEncoder(handle_unknown='ignore')
num_linear_processor = make_pipeline(StandardScaler(), SimpleImputer(strategy='mean', add_indicator=True))

linear_preprocessor = make_column_transformer((num_linear_processor, num_selector), (cat_linear_processor, cat_selector))
linear_preprocessor

ColumnTransformer(transformers=[('pipeline',
                                 Pipeline(steps=[('standardscaler',
                                                  StandardScaler()),
                                                 ('simpleimputer',
                                                  SimpleImputer(add_indicator=True))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f77e3a49d50>),
                                ('onehotencoder',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f77e3a494d0>)])

Create the Random Forest Regressor Pipeline

In [17]:
from sklearn.preprocessing import OrdinalEncoder


cat_tree_processor = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
num_tree_processor = SimpleImputer(strategy='mean', add_indicator=True)
tree_preprocessor = make_column_transformer((num_tree_processor, num_selector), (cat_tree_processor, cat_selector))
tree_preprocessor

ColumnTransformer(transformers=[('simpleimputer',
                                 SimpleImputer(add_indicator=True),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f77e3a49d50>),
                                ('ordinalencoder',
                                 OrdinalEncoder(handle_unknown='use_encoded_value',
                                                unknown_value=-1),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f77e3a494d0>)])

Create the Stacked Regressor

In [18]:
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNetCV

rf_pipeline = make_pipeline(tree_preprocessor, RandomForestRegressor(random_state=42, n_estimators=1000))
linear_pipeline = make_pipeline(linear_preprocessor, LinearRegression(fit_intercept=True))
#svr_pipeline = make_pipeline(linear_preprocessor, SVR(kernel='rbf',gamma='auto'))

estimators_1 = [('Random_Forest',rf_pipeline),('Linear_Regression',linear_pipeline)]


stacking_regressor = StackingRegressor(estimators=estimators_1, final_estimator=ElasticNetCV(l1_ratio=0.5))
stacking_regressor

StackingRegressor(estimators=[('Random_Forest',
                               Pipeline(steps=[('columntransformer',
                                                ColumnTransformer(transformers=[('simpleimputer',
                                                                                 SimpleImputer(add_indicator=True),
                                                                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f77e3a49d50>),
                                                                                ('ordinalencoder',
                                                                                 OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                                unknown_value=-1),
                                                                                 <sklearn.compos...
                                                        

Fit the Stacked Regressor model to the entire dataset

In [ ]:
from sklearn.model_selection import train_test_split
input_train, input_test, target_train, target_test = train_test_split(df_input,df_target, random_state=42)
stacking_regressor.fit(input_train,target_train)
stacking_regressor.score(input_test,target_test)

In [ ]:
df_train = input_train
df_train['predictions'] = stacking_regressor.predict(input_train)
df_train['true_times'] = target_train

df_test = input_test
df_test['predictions'] = stacking_regressor.predict(input_test)
df_test['true_times'] = target_test

ax=df_test.loc[(df_test['car']==10) & (df_test['lap']==5)].plot.scatter(x='lap_frac_only',y='true_times',c='k')
df_test.loc[(df_test['car']==10) & (df_test['lap']==5)].plot.scatter(x='lap_frac_only',y='predictions',c='r',ax=ax)
df_train.loc[(df_train['car']==10) & (df_train['lap']==5)].plot.scatter(x='lap_frac_only',y='true_times',c='gray',ax=ax)
df_train.loc[(df_train['car']==10) & (df_train['lap']==5)].plot.scatter(x='lap_frac_only',y='predictions',c='cyan',ax=ax)